In [2]:
# !pip install azureml-sdk
# %pip install --upgrade setuptools
# %pip install --upgrade azureml-sdk


**Importing the dependencies**

In [3]:
import os
import json
import requests

from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core import Workspace
from azureml.core.model import Model


In [4]:
config_file = "config.json"

# Read JSON data into a dictionary
with open(config_file, 'r') as file:
    data = json.load(file)

subscription_id = data["subscription_id"]
resource_ = data["resource_group"]  
workspace_name = data["workspace_name"]


In [5]:
print(resource_)
print(workspace_name)
# print(subscription_id)


T_work
T_workspace


In [7]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

# Perform interactive authentication
interactive_auth = InteractiveLoginAuthentication()

# Access workspace
ws = Workspace.get(name='T_workspace',
                   subscription_id='your_subscription_id',
                   resource_group='T_work',
                   auth=interactive_auth)

print(f'Workspace {ws.name} accessed successfully.')


Workspace T_workspace accessed successfully.


In [7]:
# Specify the path to your  model file
model_path = 'diabetes_model.pkl'

In [8]:
model_name='diabetes_prediction_model'

In [9]:
# Register the model in Azure Machine Learning
registered_model = Model.register(model_path=model_path, model_name=model_name, workspace=ws)

Registering model diabetes_prediction_model


In [10]:
# Create a Conda environment for your scikit-learn model
conda_env = Environment('my-conda-env')
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [11]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=conda_env)

In [12]:
# Specify deployment configuration for ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [13]:
service = Model.deploy(workspace=ws,
                       name='diabetes-prediction-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

C:\Users\mttta\AppData\Local\Temp\ipykernel_15432\3861565384.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-09-17 18:43:00+05:00 Creating Container Registry if not exists..
2024-09-17 18:53:01+05:00 Registering the environment..
2024-09-17 18:53:11+05:00 Building image..
2024-09-17 19:01:13+05:00 Generating deployment configuration..
2024-09-17 19:01:15+05:00 Submitting deployment to compute..
2024-09-17 19:01:23+05:00 Checking the status of deployment diabetes-prediction-service..
2024-09-17 19:02:42+05:00 Checking the status of inference endpoint diabetes-prediction-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [14]:
scoring_uri = service.scoring_uri

In [15]:
scoring_uri

'http://37988042-599d-43e9-9024-13d85a100f2c.southeastasia.azurecontainer.io/score'

**Cleaning up all the created resources**

In [17]:
# Delete the service

service_name = "diabetes-prediction-service"

service = ws.webservices[service_name]
service.delete()

In [18]:
# Specify the name of the registered model to delete
model_name = 'diabetes_prediction_model'

# Delete the model
registered_model = ws.models[model_name]
registered_model.delete()


In [19]:
# Delete the workspace
ws.delete(delete_dependent_resources=True)